In [ ]:
from fragalysis_api import QuickDownloader
import pandas as pd
from typing import List
print(f'Default settings are: {QuickDownloader.api_data}')

In [ ]:
url = 'https://github.com/matteoferla/munged-Fragalysis-targets/blob/main/targets.csv'

import os, re, requests

match: re.Match = re.search(r'github.com\/(?P<user>[^/]+)\/(?P<repo_name>[^/]+)\/blob/\w+\/(?P<filepath>.*)', url)
user = match.group('user')
repo_name = match.group('repo_name')
filepath = match.group('filepath')

from github import Github
repo = Github().get_user(user).get_repo(repo_name)
spreadsheet_url = repo.get_contents(filepath).download_url
response: requests.Response = requests.get(spreadsheet_url)
response.raise_for_status()
sheet_raw = response.text

from io import StringIO
import pandas as pd


projects = pd.read_csv(StringIO(sheet_raw), index_col=0)

In [ ]:
import requests

response: requests.Response = requests.get('https://fragalysis.diamond.ac.uk/api/targets/')
response.raise_for_status()
projects = pd.DataFrame(response.json()['results'])

In [ ]:
os.path.split(archive_url)[1]

In [ ]:
import os, zipfile, io

def from_download(cls, target_name, fh):
    self = cls.__new__(QuickDownloader)
    self.target_name = target_name
    self.file_url: str = ''
    self.zf = zipfile.ZipFile(fh, "r")
    return self

status = []
for archive_url, target_name in zip(projects.zip_archive, projects.title):
    response: requests.Response = requests.get(archive_url)
    # store
    filepath = os.path.join(, os.path.split(archive_url)[1])
    with open(filepath, 'wb') as fh:
        fh.write(response.content)
    zf = zipfile.ZipFile(io.BytesIO(response.content), "r")
    # check
    try:
        quick = from_download(QuickDownloader, target_name, io.BytesIO(response.content) )
        hits: pd.DataFrame = quick.to_pandas(star_dummy=True)
        block: str = quick.reference_pdbblock
    except Exception as error:
        print(target_name, error.__class__.__name__, str(error))

In [ ]:
filepaths = [os.path.join('fragalysis-downloads', fn) for fn in os.listdir('fragalysis-downloads') if '.zip' in fn]

statuses = []
for filepath in filepaths:
    target_name = os.path.splitext( os.path.split(filepath)[1] )[0]
    status = dict(name=target_name, filepath=filepath)
    with open(filepath, 'rb') as fh:
        quick = from_download(QuickDownloader, target_name, fh )
        for item in ('metadata', 'reference', 'combined'):
            for fileinfo in quick.zf.infolist():
                if item in fileinfo.filename:
                    status[item] = True
                    break
            else:
                status[item] = False
        statuses.append(status)
        
status = pd.DataFrame(statuses)

In [ ]:
status